In [19]:
!pip install keras_tuner
!pip install wget
!pip install tensorflow_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
import os, warnings
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
import tensorflow as tf
from keras.utils import np_utils
import keras_tuner as kt
from keras.applications.vgg16 import VGG16
import tensorflow_datasets as tfds
import glob
import os

# Helper Functions

In [13]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(10,10))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

# Project 2 - Veggie Classification

For this assignment you'll need to classify some images of vegetables. 

## Parts

Please do two separate classifications:
<ol>
<li> First, create a model from scratch. 
<li> Use transfer learning to use a pretrained model of your choice, adapted to this data. 
</ol>

There won't be an explicit evaluation of accuracy, but you should take some steps to make each model as accurate as you reasonably can, any tuning option is fair game. Along with that, please structure it into a notebook that is well structured and clear that explains what you did and found. Think about:
<ul>
<li> Sections and headings. 
<li> A description of the approach taken (e.g. what did you do to determine size, tune, evaluate, etc...)
<li> Visualization of some important things such as a confusion matrix and maybe some images. 
<li> Results, mainly focused on the scoring of the test data. 
</ul>

The descriptions and explainations should highlight the choices you made and why you made them. Figure up to about a page or so worth of text total, explain what happened but don't write an essay. 

## Deliverables

Please sumbmit a link to your github, where everyhting is fully run with all the outputs showing on the page. As well, in the notebook please add some kind of switch controlled by a variable that will control if the notebook runs to train the model or to load the model in from weights - so I can download it and click run all, it will load the saved weights, and predict.

### Dataset

The code in the start of this notebook will download and unzip the dataset, and there is also a simple example of creating datasets. You can change the dataset bit to use a different approach if you'd like. The data is already split into train, validation, and test sets. Please treat the separate test set as the final test set, and don't use it for any training or validation. Each folder name is its own label.

### Evaluation

Marking will be based on the following:
<ul>
<li> Models are cretaed, tuned, and effective at classifying the data: 40%
<li> Descriptions and explanations of the approach taken: 20%
<li> Code is well structured and clear: 20%
</ul>

Overall the marking is pretty simple and direct, walk through the process of predicting the veggies, explain what you did, and show the results. If you do that, it'll get a good mark.

### Tips

Some hints that may be helpful to keep in mind:
<ul>
<li> The data is pretty large, so you'll want to use datasets rather than load everything into memory. The Keras docs have a few examples of different ways to load image data, our examples showed image generators and the image from directory datasets.  
<li> Be careful of batch size, you may hit the colab limits. 
<li> You'll want to use checkpoints so you can let it train and pick up where you left off.
<li> When developing, using a smaller dataset sample is a good idea. These weights could also be saved and loaded to jump start training on the full data. 
<li>

### Training Switch

In [3]:
training = True

### Download and Unzip Data

In [4]:
if training:
    def bar_custom(current, total, width=80):
        print("Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total))
    import wget
    import zipfile
    zip_name = "train.zip"

    url = "https://jrssbcrsefilesnait.blob.core.windows.net/3950data1/vegetable_image_dataset.zip"

    if not os.path.exists(zip_name):
        wget.download(url, zip_name, bar=bar_custom)

    with zipfile.ZipFile(zip_name, 'r') as zip_ref:
        zip_ref.extractall()

Streaming output truncated to the last 5000 lines.
Downloading: 92% [519086080 / 560031432] bytes
Downloading: 92% [519094272 / 560031432] bytes
Downloading: 92% [519102464 / 560031432] bytes
Downloading: 92% [519110656 / 560031432] bytes
Downloading: 92% [519118848 / 560031432] bytes
Downloading: 92% [519127040 / 560031432] bytes
Downloading: 92% [519135232 / 560031432] bytes
Downloading: 92% [519143424 / 560031432] bytes
Downloading: 92% [519151616 / 560031432] bytes
Downloading: 92% [519159808 / 560031432] bytes
Downloading: 92% [519168000 / 560031432] bytes
Downloading: 92% [519176192 / 560031432] bytes
Downloading: 92% [519184384 / 560031432] bytes
Downloading: 92% [519192576 / 560031432] bytes
Downloading: 92% [519200768 / 560031432] bytes
Downloading: 92% [519208960 / 560031432] bytes
Downloading: 92% [519217152 / 560031432] bytes
Downloading: 92% [519225344 / 560031432] bytes
Downloading: 92% [519233536 / 560031432] bytes
Downloading: 92% [519241728 / 560031432] bytes
Downloadi

In [5]:
# Generate Datasets - you can change this if desired
# ENSURE FILE PATHS MATCH CORRECTLY
if training:
    IMAGE_SIZE=(224,224)
    train_dir='Vegetable Images/train'
    val_dir='Vegetable Images/validation'
    batch_size = 16
    BUFFER = 150
    SEED = 42

    # Load training data
    train_ds = image_dataset_from_directory(
        train_dir,
        label_mode='categorical',
        image_size = IMAGE_SIZE,
        batch_size=batch_size
    )

    val_ds = image_dataset_from_directory(
        val_dir,
        label_mode='categorical',
        image_size = IMAGE_SIZE,
        batch_size = batch_size,
    )

Found 15000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.


## Data Preparation

In [ ]:
if training:
    plt.imshow(train_ds.as_numpy_iterator().next()[0][4].astype("int"))

In [6]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.legend()
  plt.grid(True)
  plt.show()

def plot_acc(history):
  plt.plot(history.history['accuracy'], label='accuracy')
  plt.plot(history.history['val_accuracy'], label='val_accuracy')
  plt.legend()
  plt.grid(True)
  plt.show()

In [55]:
if training:
  BASE_EPOCHS = 20
  BATCH_SIZE = 10000
  BASE_PATIENCE = 5
  MIN_DELTA = .02
  MONITOR = "val_accuracy"
  MODE = "max"
  WEIGHT_PATH = "weights/"

  try:
    import google.colab
    IN_COLAB = True
  except:
    IN_COLAB = False

  if IN_COLAB:
    BASE_EPOCHS = 75
    BATCH_SIZE = 10000
    BASE_PATIENCE = 10

  acc = keras.metrics.CategoricalAccuracy(name="accuracy")
  pre = keras.metrics.Precision(name="precision")
  rec = keras.metrics.Recall(name="recall")
  metric_list = [acc, pre, rec]

  callback = tf.keras.callbacks.EarlyStopping(monitor=MONITOR, patience=BASE_PATIENCE, restore_best_weights=True, min_delta=MIN_DELTA, mode=MODE)
  checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(WEIGHT_PATH + "weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_accuracy', verbose=2, save_best_only=True, save_weights_only=True, mode='max', save_freq='epoch')

  callbacks = [callback,checkpoint_callback]

In [ ]:
train_ds = train_ds.shuffle(buffer_size=BUFFER)

In [ ]:
TPAIN = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=TPAIN)
val_ds = val_ds.prefetch(buffer_size=TPAIN)

## Custom Model Training

In [8]:
if training:
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(64, (3,3), padding="same",  input_shape=(224, 224, 3)),
      tf.keras.layers.LeakyReLU(),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(128, (3,3), padding="same"),
      tf.keras.layers.LeakyReLU(),
      tf.keras.layers.MaxPooling2D(),
      # tf.keras.layers.Conv2D(256, (3,3), padding="same"),
      # tf.keras.layers.LeakyReLU(),
      # tf.keras.layers.MaxPooling2D(),
      # tf.keras.layers.Conv2D(512, (3,3), padding="same"),
      # tf.keras.layers.LeakyReLU(),
      # tf.keras.layers.MaxPooling2D(),
      # tf.keras.layers.Dropout(.2),

      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, kernel_regularizer="l2"),
      tf.keras.layers.LeakyReLU(),
      tf.keras.layers.Dropout(.3),
      tf.keras.layers.Dense(128, kernel_regularizer="l2"),
      tf.keras.layers.LeakyReLU(),
      tf.keras.layers.Dense(15, activation="softmax")
  ])
  model.compile(
    #optimizer=tf.optimizers.Adam(learning_rate=0.001),
    optimizer="rmsprop",
    #loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    loss="categorical_crossentropy",
    metrics=['accuracy']
    )
  model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 224, 224, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 112, 112, 128)     0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                    

In [9]:
# Train
if training:
  if os.path.exists(WEIGHT_PATH):
    list_of_files = glob.glob(WEIGHT_PATH+'*.hdf5') # gets all model weights files
    latest_file = max(list_of_files, key=os.path.getctime)
    if not latest_file:
      model.load_weights(latest_file)
  log_m1 = model.fit(
      train_ds,
      validation_data=val_ds,
      epochs=BASE_EPOCHS,
      callbacks=callbacks
  )
  plot_loss(log_m1)
  plot_acc(log_m1)
  model.save('Scratch')

Epoch 1/75
938/938 [==============================] - 114s 108ms/step - loss: 556.1024 - accuracy: 0.3139 - val_loss: 178.2698 - val_accuracy: 0.3277
Epoch 2/75
938/938 [==============================] - 100s 107ms/step - loss: 184.2050 - accuracy: 0.3512 - val_loss: 107.1109 - val_accuracy: 0.3633
Epoch 3/75
938/938 [==============================] - 100s 106ms/step - loss: 170.3951 - accuracy: 0.3504 - val_loss: 110.0277 - val_accuracy: 0.2930
Epoch 4/75
938/938 [==============================] - 100s 106ms/step - loss: 167.1455 - accuracy: 0.3426 - val_loss: 88.0934 - val_accuracy: 0.3767
Epoch 5/75
938/938 [==============================] - 98s 105ms/step - loss: 131.9743 - accuracy: 0.3519 - val_loss: 77.1590 - val_accuracy: 0.3890
Epoch 6/75
938/938 [==============================] - 99s 105ms/step - loss: 118.3103 - accuracy: 0.3499 - val_loss: 61.5347 - val_accuracy: 0.4000
Epoch 7/75
398/938 [===========>..................] - ETA: 53s - loss: 121.2165 - accuracy: 0.3464

KeyboardInterrupt: ignored

## Transfer Learning Model

In [41]:
#get VGG16 NN
if training:
  base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
  base_model.trainable = False ## Not trainable weights

  # Add Dense Stuff
  flatten_layer = Flatten()
  dense_layer_1 = Dense(512, activation='relu', kernel_regularizer='l2', bias_regularizer='l2')
  drop_layer_1 = Dropout(.2)
  dense_layer_2 = Dense(256, activation='relu', kernel_regularizer='l2', bias_regularizer='l2')
  prediction_layer = Dense(15, activation='softmax')

  tlModel = Sequential([
      base_model,
      flatten_layer,
      dense_layer_1,
      drop_layer_1,
      dense_layer_2,
      prediction_layer
  ])
  tlModel.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=['accuracy'])
  tlModel.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dense_6 (Dense)             (None, 512)               12845568  
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 15)                3855      
                                                                 
Total params: 27,695,439
Trainable params: 12,980,751


In [71]:
if training:
  if os.path.exists(WEIGHT_PATH):
    list_of_files = glob.glob(WEIGHT_PATH+'*.hdf5') # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getctime)
    if not latest_file:
      tlModel.load_weights(latest_file)
  log_m2 = tlModel.fit(
      train_ds,
      validation_data=val_ds,
      epochs=BASE_EPOCHS,
      callbacks=callbacks
  )
  plot_loss(log_m2)
  plot_acc(log_m2)
  tlModel.save('Transfer')

Epoch 1/75
 38/938 [>.............................] - ETA: 1:02 - loss: 0.7167 - accuracy: 0.9145

KeyboardInterrupt: ignored

## Test Best Models and Illustrate Results

In [73]:
test_dir='Vegetable Images/test'
label_walk = os.walk(test_dir)
labels = []
for path, directories, files in label_walk:
     for directory in directories:
          labels.append(directory)
labels.sort()

test_ds = tf.keras.utils.image_dataset_from_directory(test_dir, seed=42, image_size = IMAGE_SIZE, batch_size = batch_size, shuffle=True, labels='inferred', label_mode="categorical", class_names=labels)

# test_ds = image_dataset_from_directory(
#     test_dir,
#     label_mode='categorical',
#     image_size = IMAGE_SIZE,
#     batch_size = batch_size,
# )


Found 3000 files belonging to 15 classes.


In [ ]:
if training == False:
    model = keras.models.load_model('Scratch')
    tlModel = keras.models.load_model('Transfer')

In [74]:
# badModelPredictionResults = model.predict(test_ds)

transferModelPredictionResults = tlModel.predict(test_ds)

188/188 [==============================] - 12s 64ms/step


In [67]:
# y = np.concatenate([y for x, y in test_ds], axis=0)
for images, labels in test_ds.take(1):
  # print(labels)
  numpy_images = images.numpy() 
  numpy_labels = labels.numpy()

In [68]:
numpy_labels

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

# Comparison

In [ ]:
labels = []
for i in range(test_ds.__len__()):
  labels.append(list(test_ds)[i][1])

In [ ]:
plot_cm(labels,transferModelPredictionResults)

In [ ]:
test_predictions_baseline = model.predict(test_features, batch_size=BATCH_SIZE)
baseline_results = model.evaluate(test_features, test_labels,batch_size=BATCH_SIZE, verbose=0)

In [ ]:
for name, value in zip(model.metrics_names, baseline_results):
    print(name, ': ', value)
plot_cm(test_labels, test_predictions_baseline)